# Ruptura

In [ ]:
import pandas as pd
import numpy as np
import json
from datetime import timedelta

def read_data(store_name):
    try:
        stock_df = pd.read_csv('storage_data/' + store_name + '.csv', index_col=0)
        sales_df = pd.read_csv('data/' + store_name + '.csv', index_col=0)

        stock_df.index = pd.to_datetime(stock_df.index)
        sales_df.index = pd.to_datetime(sales_df.index)
    
        return stock_df, sales_df
    except:
        print('Error while opening %s data' % (store_name))
        return None, None

In [ ]:
def set_nan_values(stock_df):  
    # Considera-se como uma data em que não há contagem a data onde o estoque de todos os produtos é 0
    stock_df[stock_df.index.day_of_week == 6] = np.nan # Não há contagem no domingo
    stock_df[stock_df.sum(axis=1) == 0] = np.nan # Pegar as outras datas em que não houveram contagem
    # Desconsiderar as datas em que não possuimos dados
    first_date_w_data = stock_df[stock_df.sum(axis=1) > 0].index[0]
    stock_df[:first_date_w_data] = 0 
    
    return stock_df

In [ ]:
def get_missing_dates(stock_df):
    na_rows = stock_df.isna().any(axis=1)
    missing_dates = na_rows[na_rows == True].index
    
    return missing_dates

In [ ]:
def fix_stock(stock_df, sales_df):
    stock_df = set_nan_values(stock_df)
    missing_dates = get_missing_dates(stock_df)
    
    for date in missing_dates:
        stock_df.loc[date - timedelta(days=1) : date] = stock_df.loc[date - timedelta(days=1) : date].fillna(method='ffill')
        stock_df.loc[date] -= sales_df.loc[date - timedelta(days=1)]
    
    # Corrigir estoque negativo
    return stock_df.applymap(lambda x : x if x > 0 else 0)

In [ ]:
stores = ['Ico', 'Jacobina', 'Lagoa', 'Campina Grande I', 'Bayeux',
       'Esperanca', 'Irece I', 'Mangabeira Shopping', 'Patos I', 'Matriz',
       'Patos II', 'Campina Grande III', 'Itambe', 'Campina Grande II',
       'Mangabeira', 'Aristides Lobo', 'Sape', 'Sta. Rita', 'Iguatu',
       'Pombal', 'Itaporanga', 'Mamanguape', 'Sousa I', 'Timbauba',
       'Joao Dourado', 'Goiana', 'Cuite', 'Queimadas', 'Irece II',
       'Solanea', 'Guarabira', 'Cabedelo', 'Alagoa Grande', 'AREIA',
       'Bancarios', 'Sousa II', 'Barao', 'Soledade', 'Guarabira II',
       'Goiana 2', 'Geisel', 'Picui', 'Alhandra', 'Itabaiana',
       'Lagoa Seca', 'Campina Grande IV', 'Monteiro', 'Patos III',
       'Pianco', 'Sto. Elias', 'Monteiro 2', 'Caruaru',
       'Manaira', 'CABEDELO BR', 'Patos IV']

i = 0
for store in stores:
    i += 1
    print('(%d/%d)' % (i, len(stores)))
    print('Reading %s data' % (store))
    stock_df, sales_df = read_data(store)
    before = stock_df.sum(axis=1)
    print('Fixing %s data' % (store))
    stock_df = fix_stock(stock_df, sales_df)
    print('Saving fixed %s data' % (store))
    stock_df.to_csv('storage_data/' + store + '.csv')